In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv("./properatti.csv")
print(data.shape)
data.head(2)

(121220, 26)


,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.0,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...


In [13]:
print(data.shape) # tamaño del DataFrame
print(data.info())

(121220, 26)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  121220 non-null  int64  
 1   operation                   121220 non-null  object 
 2   property_type               121220 non-null  object 
 3   place_name                  121197 non-null  object 
 4   place_with_parent_names     121220 non-null  object 
 5   country_name                121220 non-null  object 
 6   state_name                  121220 non-null  object 
 7   geonames_id                 102503 non-null  float64
 8   lat-lon                     69670 non-null   object 
 9   lat                         69670 non-null   float64
 10  lon                         69670 non-null   float64
 11  price                       100810 non-null  float64
 12  currency                    100809 non-null  object 
 13  p

In [14]:
#tipos de datos de cada columna
data.dtypes

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

In [15]:
#surface_covered_in_m2, price_usd_per_m2
data.surface_covered_in_m2.describe()

data.price_usd_per_m2.describe()


count     68617.000000
mean       2160.086916
std        2759.288621
min           0.600000
25%        1218.181818
50%        1800.000000
75%        2486.411765
max      206333.333333
Name: price_usd_per_m2, dtype: float64

La libreria missingno proporciona una serie de visualizaciones para comprender la presencia y distribución de datos faltantes dentro de un DataFrame

In [ ]:
import missingno as msno # proporciona una serie de visualizaciones para comprender la presencia y distribución de datos faltantes dentro de un DataFrame

data.isna().sum()

msno.bar(data, color="dodgerblue", sort="ascending", fontsize=12, figsize=(18,8)) # gráfico simple donde cada barra representa una columna dentro del marco de datos. La altura de la barra indica qué tan completa está esa columna, es decir, cuántos valores no nulos están presentes
msno.heatmap(data, fontsize=12, figsize=(18,8)) #el mapa de calor se puede utilizar para identificar si existe una relación en presencia de valores nulos entre cada una de las columnas
msno.dendrogram(data, fontsize=12, figsize=(18,8))

Teniendo en Cuenta los análisis anteriores, se opta por eliminar las siguientes columnas:

Categoricas (practicamente todas las observaciones son diferentes)
1) Unnamed: 0
2) operation
3) place_with_parent_names	
4) properati_url
5) description	
6) title
7) image_thumbnail
8) country_name (todas las observaciones son de Argentina)

Cuantitativas con muy pocos datos (<40%)

9) floor
10) expenses


Hay 5 variables que tienen la misma información, con coeficiente de correlación de "1", será seleccionada price_usd_per_m2 ya que da amyor información y estandar (USD/m2). Por lo tanto se eliminan las siguientes columnas

price                         
currency                       
price_aprox_local_currency
price_aprox_usd

In [ ]:
# data_1 tiene el DataFrame despues de eliminar columnas 

data_1 = data.drop(["Unnamed: 0", "operation", "place_with_parent_names", "geonames_id", "lat", "lon", "properati_url", "description", "title",  "image_thumbnail", "floor", "expenses", "currency", "country_name", "lat-lon", "price", "price_aprox_local_currency", "rooms"], axis=1)
data_1.head(3)


Teniendo en cuenta que nuestra variable objetivo es price_usd_per_m2, calcularemos algunas observaciones NaN a partir de price_aprox_usd y surface_total_in_m2

In [ ]:
#data_5=data_1.apply(lambda x: x.fillna(value={"price_usd_per_m2": data_1["price_aprox_usd"]/data_1["surface_total_in_m2"]}))
#data_5.price_usd_per_m2.isna().count()
#data_1.price_usd_per_m2.isna().count()

print("El número de Nan para price_price_usd_per_m2 es: ", data_1.price_usd_per_m2.isna().sum())
print("El número de Nan para price_aprox_usd es: ", data_1.price_aprox_usd.isna().sum())
print("El número de Nan para surface_total_in_m2 es: ", data_1.surface_total_in_m2.isna().sum())

data_2=data_1.apply(lambda x: x.fillna(value={"price_usd_per_m2": data_1["price_aprox_usd"]/data_1["surface_total_in_m2"]}))

print("El número de Nan para price_price_usd_per_m2 despues de aplicar la función es: ", data_2.price_usd_per_m2.isna().sum())



Se concluye que tenemos la sumatoria de NaN total en price_usd_per_m2 debido a la falta del dato en el númerador (price_aprox_usd) o denominador (surface_total_in_m2)

Eliminamos observaciones NaN en la variable "Y" (price_usd_per_m2)

In [ ]:
#data_2 = data_1.dropna(subset = ["price_usd_per_m2"])
#data_2.price_usd_per_m2.isna().count()

mask_isna = data_1.price_usd_per_m2.isna() == False
data_2 = data_1[mask_isna]

data_2.head(5)

Visualizar datos extremos

In [ ]:
import seaborn as sns

fig, axes = plt.subplots(2, 2, figsize = (18,6))
sns.boxplot(x=data_2["price_usd_per_m2"], ax=axes[0,0])
sns.boxplot(x=data_2["surface_covered_in_m2"], ax=axes[0,1])
sns.boxplot(x=data_2["surface_covered_in_m2"], ax=axes[1,0])
sns.scatterplot(data=data_2, x="surface_covered_in_m2", y="price_usd_per_m2", ax=axes[1,1])
plt.show

In [ ]:
#Analisis de cuartiles para precio en USD por m2


Q1_precio_usdm2 = data_2["price_usd_per_m2"].quantile(0.25)
Q3_precio_usdm2 = data_2["price_usd_per_m2"].quantile(0.75)
print("los cuartiles para para precio en USD por m2 son: ", round(Q1_precio_usdm2), round(Q3_precio_usdm2))

IQR_pusd = Q3_precio_usdm2-Q1_precio_usdm2 

lim_inf_pusd = Q1_precio_usdm2  - 2*IQR_pusd
lim_sup_pusd = Q3_precio_usdm2 + 2*IQR_pusd

print("Los limites para para precio en USD por m2 son: ", round(lim_inf_pusd), round(lim_sup_pusd))

In [ ]:
##Eliminar datos atipicos

# Precio en dolares por en m2

mask_precio_m2 = data_2.price_usd_per_m2<= 5023  # con esta mascara se eliminan 873 observaciones
mask_isna_precio_m2 = data_2.price_usd_per_m2.isna() == True

import pandas as pd
data_precio_sin_outlier = data_2[mask_precio_m2 | mask_isna_precio_m2]
print("El tamaño DataFrame despues de eliminar los outlier es: ",data_precio_sin_outlier.shape)

In [ ]:
import pandas as pd
import seaborn as sns


plt.figure(figsize=(16,8))
plt.subplot(2,2,1)
g1=sns.histplot(data_2["price_usd_per_m2"], kde=True)
g1.set(xlabel ="PRECIO EN USD/m2 - antes de la eliminación de puntos extremos", ylabel = "Conteo")
plt.subplot(2,2,2)
g2=sns.boxplot(x=data_2["price_usd_per_m2"])
g2.set(xlabel ="PRECIO EN USD/m2 - antes de la eliminación de puntos extremos")
plt.subplot(2,2,3)
g3=sns.histplot(data_precio_sin_outlier["price_usd_per_m2"], kde=True)
g3.set(xlabel ="PRECIO EN USD POR m2 - Despues de la eliminación de puntos extremos", ylabel = "Conteo")
plt.subplot(2,2,4)
g4=sns.boxplot(x=data_precio_sin_outlier["price_usd_per_m2"])
g4.set(xlabel ="PRECIO EN USD/m2 - Despues de la eliminación de puntos extremos")
plt.show()

In [ ]:
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize = (12,6))

s_p = sns.histplot(data=data_scm2, x = "surface_covered_in_m2", hue="property_type", ax=axes[0])
s_p.set(xlim = (0, 1000))

p_p = sns.histplot(data=data_usdm2, x = "price_usd_per_m2", hue="property_type", ax=axes[1])
p_p.set(xlim = (0, 10000))

In [ ]:
#Histograma
%matplotlib inline

n_bins = 50

fig, axs = plt.subplots(1,2, figsize = (15,10))
axs[0].hist(data_scm2.surface_covered_in_m2, bins = n_bins)
axs[1].hist(data_usdm2.price_usd_per_m2, bins = n_bins)
#axs[1,0].hist(data.price_usd_per_m2)
#axs[1,1].hist(data.price_per_m2)



Segun estos análisis la Superficie mayores a 1000m2  y los precios en dolares por m2 mayores a 10000 son puntos extremos. 
Esto no quiere decir que no sean "reales", pero van a sesgar seguramento futuros modelos en el TP2
Teniendo en cuenta estos limites, tenemos 379 observaciones extremas para "surface_covered_in_m2" y 873 para "price_usd_per_m2"

¿Que se hacemos con esas observaciones?

In [ ]:
msno.bar(data_1, color="dodgerblue", sort="ascending", fontsize=12, figsize=(18,8))
msno.heatmap(data_1, fontsize=12, figsize=(18,8)) #el mapa de calor se puede utilizar para identificar si existe una relación en presencia de valores nulos entre cada una de las columnas
msno.dendrogram(data_1, fontsize=12, figsize=(18,8))

In [ ]:
data.price.count()
data.price_aprox_usd.count()
data.price_aprox_local_currency.count()


data_1.surface_total_in_m2.count()
data_1.surface_covered_in_m2.count()

In [ ]:
#mask_100m2 = data.surface_total_in_m22<= 1000
mask_isna_superficie_total = data_1.surface_total_in_m2.isna() == False
data_2 = data_1[mask_isna_superficie_total]

data_2.shape

data_2.head(5)

In [ ]:
prome_agrup_tipo_place = data_2.groupby(["property_type", "place_name"]).mean()
prome_agrup_tipo_place

In [ ]:
data_4=data_1.groupby(["property_type", "place_name"]).apply(lambda x: x.fillna(value={"price_usd_per_m2": x["price_usd_per_m2"].mean()}))

print("número de NaN despues de aplicar la función a price_usd_per_m2: ",data_4.price_usd_per_m2.isna().count())
print("número de NaN antes de aplicar la función a price_usd_per_m2: ", data_1.price_usd_per_m2.isna().count())
print("Diferencia en el número de NaN: ", data_1.price_usd_per_m2.isna().count() - data_4.price_usd_per_m2.isna().count())

data_4.head(5)


In [ ]:
p_p = sns.histplot(data=data_4, x = "price_usd_per_m2", hue="property_type")
p_p.set(xlim = (0, 8000))